# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

%load_ext dotenv
%dotenv 


In [2]:
import dask.dataframe as dd

c:\Users\darli\anaconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [6]:
import os
import sys
from glob import glob

# Write your code below.

os.getenv('PRICE_DATA')
sys.path.append(os.getenv('PRICE_DATA'))

stock_prices_parquet = glob(os.path.join(os.getenv('PRICE_DATA'),"*","*","*.parquet"))

stock_prices_parquet

['../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2004\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2004\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2005\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2005\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2006\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2006\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2007\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2007\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2008\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2008\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_200

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [21]:
# Write your code below.

stock_prices_dd = dd.read_parquet(stock_prices_parquet).set_index("ticker")
print(stock_prices_dd)

print ("##############################################################################")
#to avoid errors when computing lags for the initial row:
meta = stock_prices_dd._meta.assign(
    Close_lag_1=0.0,
    Adj_Close_lag_1=0.0
)

#this is for the lags 
dd_shift = stock_prices_dd.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1),
        Adj_Close_lag_1 = x['Adj Close'].shift(1)
    ),
    meta=meta
)

#this is for the returns and hi_lo_range
dd_feat = dd_shift.assign(
    returns = lambda x: (x['Close'] / x['Close_lag_1']) - 1,
    hi_lo_range = lambda x: x['High'] - x['Low']
)

result = dd_feat.compute()
print(result.head())

Dask DataFrame Structure:
                          Date     Open     High      Low    Close Adj Close   Volume  source   Year
npartitions=90                                                                                      
ACN             datetime64[ns]  float64  float64  float64  float64   float64  float64  object  int32
ALDX                       ...      ...      ...      ...      ...       ...      ...     ...    ...
...                        ...      ...      ...      ...      ...       ...      ...     ...    ...
ZIXI                       ...      ...      ...      ...      ...       ...      ...     ...    ...
ZIXI                       ...      ...      ...      ...      ...       ...      ...     ...    ...
Dask Name: setindex, 2 expressions
Expr=SetIndex(frame=ReadParquetFSSpec(e42e6ec), _other='ticker', options={})
##############################################################################
             Date   Open   High    Low  Close  Adj Close      Volume   sourc

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [27]:
# Write your code below.

#we already converted to pandas dataframe in the step above and we have it as a "result" pandas dataframe, because we wanted to visualize the result

print(type(result))

#now the missing part is to add that new feature containing the moving average of returns using a windows of 10 days so:

#I tried to use the following but I realize then I could combine between tickers
#result['moving_average_returns_10d']= result['returns'].rolling(10).mean()
#result.head(20)

#Improved version consider reseting the index for each ticker, and calculating the moving average for the last 10 days, 
# but even with just 1 record for a ticker

result['moving_average_returns_10d'] = (
    result.groupby('ticker')['returns']
    .rolling(window=10, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)
result.head(20)

<class 'pandas.core.frame.DataFrame'>


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,moving_average_returns_10d
ticker,,,,,,,,,,,,,,
ACN,2001-07-19,15.10,15.29,15.00,15.17,11.404394,34994300.0,ACN.csv,2001,NaN,NaN,NaN,0.290000,NaN
ACN,2001-07-20,15.05,15.05,14.80,15.01,11.284108,9238500.0,ACN.csv,2001,15.17,11.404394,-0.010547,0.250000,-0.010547
ACN,2001-07-23,15.00,15.01,14.55,15.00,11.276587,7501000.0,ACN.csv,2001,15.01,11.284108,-0.000666,0.460000,-0.005607
ACN,2001-07-24,14.95,14.97,14.70,14.86,11.171341,3537300.0,ACN.csv,2001,15.00,11.276587,-0.009333,0.270000,-0.006849
ACN,2001-07-25,14.70,14.95,14.65,14.95,11.238999,4208100.0,ACN.csv,2001,14.86,11.171341,0.006057,0.300000,-0.003623
ACN,2001-07-26,14.95,14.99,14.50,14.50,10.900705,6335300.0,ACN.csv,2001,14.95,11.238999,-0.030100,0.490000,-0.008918
ACN,2001-07-27,14.51,14.59,14.50,14.51,10.908223,3524000.0,ACN.csv,2001,14.50,10.900705,0.000690,0.090000,-0.007317
ACN,2001-07-30,14.50,14.78,14.50,14.70,11.051059,3654300.0,ACN.csv,2001,14.51,10.908223,0.013094,0.280000,-0.004401
ACN,2001-07-31,14.71,15.01,14.60,14.96,11.246520,1429000.0,ACN.csv,2001,14.70,11.051059,0.017687,0.410000,-0.001640


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

In [35]:
#Many times is not necessary to convert to pandas, in this case we could try to calculate the moving average as many pandas features are in dask, 
# To simplify I am not going to group by ticker this time just to probe the statement.

dd_feat['moving_average_returns_10d'] = dd_feat['returns'].rolling(window=10, min_periods=1).mean()

result = dd_feat.compute()
result = result.reset_index()
result.head(20)

#Dask will be better for large datasets when we will need to distribute the computing across multiple nodes
#so we were not be limited to memory ram from just one node

,ticker,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,moving_average_returns_10d
0,ACN,2001-07-19,15.10,15.29,15.00,15.17,11.404394,34994300.0,ACN.csv,2001,NaN,NaN,NaN,0.290000,NaN
1,ACN,2001-07-20,15.05,15.05,14.80,15.01,11.284108,9238500.0,ACN.csv,2001,15.17,11.404394,-0.010547,0.250000,-0.010547
2,ACN,2001-07-23,15.00,15.01,14.55,15.00,11.276587,7501000.0,ACN.csv,2001,15.01,11.284108,-0.000666,0.460000,-0.005607
3,ACN,2001-07-24,14.95,14.97,14.70,14.86,11.171341,3537300.0,ACN.csv,2001,15.00,11.276587,-0.009333,0.270000,-0.006849
4,ACN,2001-07-25,14.70,14.95,14.65,14.95,11.238999,4208100.0,ACN.csv,2001,14.86,11.171341,0.006057,0.300000,-0.003623
5,ACN,2001-07-26,14.95,14.99,14.50,14.50,10.900705,6335300.0,ACN.csv,2001,14.95,11.238999,-0.030100,0.490000,-0.008918
6,ACN,2001-07-27,14.51,14.59,14.50,14.51,10.908223,3524000.0,ACN.csv,2001,14.50,10.900705,0.000690,0.090000,-0.007317
7,ACN,2001-07-30,14.50,14.78,14.50,14.70,11.051059,3654300.0,ACN.csv,2001,14.51,10.908223,0.013094,0.280000,-0.004401
8,ACN,2001-07-31,14.71,15.01,14.60,14.96,11.246520,1429000.0,ACN.csv,2001,14.70,11.051059,0.017687,0.410000,-0.001640
9,ACN,2001-08-01,15.00,15.50,14.90,15.50,11.652478,2087900.0,ACN.csv,2001,14.96,11.246520,0.036096,0.600000,0.002553


## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.